# Task Sequencer

#### <i>"Sometimes when you innovate, you make mistakes. It is best to admit them quickly, and get on with improving your other innovations."<i> -Steve Jobs

This is a tool which will calculate the optimal order to approach tasks in a project where there are several different tasks that need to be done that have a certain probability of failing and will take a certain time to complete.

When we are uncertain about the feasibility of a project, we should strive to get as much information about the difficulty as quickly as possible. This suggests starting with tasks with a high failure rate.

The idea behind this approach is and some other useful information is discussed here. https://cs.stanford.edu/~jsteinhardt/ResearchasaStochasticDecisionProcess.html

In [2]:
import ipywidgets
import ipysheet
import pandas as pd
from itertools import permutations
import numpy as np
import warnings
import matplotlib.pyplot as plt
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['lines.linewidth'] = 2.0
from IPython.display import Markdown, display, clear_output, FileLink


def printmd(string):
    display(Markdown(string))

def df_from_sheet(sheet):
    '''Converts the sheet to a compact dataframe'''
    df = ipysheet.to_dataframe(sheet)
    df = fill_blanks(df)
    df = trim_df(df)
    df = switch_from_names(df)
    df = nums_to_float(df)
    df = switch_to_names(df)
    return df

def switch_to_names(df):
    '''Converts indicies in the dependency cols
    to their corresponding task names'''
    df.iloc[:,3:] = df.iloc[:,3:].replace(
        dict(zip(df.index + 1, df['task'])))
    return df

def switch_from_names(df):
    '''Converts the task names  in the dependency 
    cols to their coresponding indicies'''
    df.iloc[:,3:] = df.iloc[:,3:].replace(
        dict(zip(df['task'], df.index + 1)))
    return df

def fill_blanks(df):
    '''Replaces NaN and empty values with 0s'''
    df.fillna(0, inplace=True)
    df.replace(["", "0"], 0, inplace=True)
    return df

def trim_df(df):
    '''Removes rows and columns that dont contain
    any information'''
    zero_row_bool = (df.values==0).all(axis=1)
    zero_row_loc = np.where(zero_row_bool)[0]
    if zero_row_loc.size == 0:
        first_zero_row=df.shape[0]
    else:
        first_zero_row = zero_row_loc[0]
    zero_col_bool = (df.values==0).all(axis=0)
    zero_col_loc = np.where(zero_col_bool)[0]
    if zero_col_loc.size == 0:
        first_zero_col = df.shape[1]
    else:
        first_zero_col = zero_col_loc[0]
    return df.iloc[:first_zero_row,:first_zero_col]
    
def nums_to_float(df):
    '''Converts all numerical entries
    to floats'''
    df.iloc[:,1:] = df.iloc[:,1:].astype(float)
    return df

def build_sheet(task_report="task_sheet.csv",
               rows=10, cols=10):
    '''Creates the task dependency sheet'''
    labels = (['task', 'time', 'probability'] 
              + ["d" + str(i + 1) 
                 for i in range(cols - 3)])
    sheet = sheet_from_df(task_report, rows, 
                          cols, labels)
    if sheet:
        return sheet
    sheet = sheet_from_scratch(rows, cols, labels)
    return sheet

def sheet_from_df(task_report, rows, 
                  cols, labels):
    '''Returns a dataframe from task_report if
    task_report is in the working directory. Otherwise
    returns None.'''
    try:
        sheet = ipysheet.sheet(
            rows=rows,
            columns=cols,
            column_headers=labels)
        df = pd.read_csv(task_report)
        df = rid_zeros(df)
        data = np.empty([rows, cols], dtype=object)
        data[:df.shape[0], :df.shape[1]] = df.values
        ipysheet.cell_range(data)
        return sheet
    except:
        return None

def rid_zeros(df):
    '''Converts the task names  in the dependency 
    cols to their coresponding indicies'''
    df.iloc[:,3:] = df.iloc[:,3:].replace({'0.0' : None})
    return df
    
    
def sheet_from_scratch(rows, cols, labels):
    '''Generates a sheet with all empty values'''
    sheet = ipysheet.sheet(
        rows=rows,
        columns=cols,
        column_headers=labels)
    data = [['' for i in range(cols)] 
            for i in range(rows)]
    ipysheet.cell_range(data)
    return sheet

def gen_perms(df):
    '''Creates a list of all permutations of the
    rows of the dataframe (indexing starting at 1)'''
    rows=df.shape[0]
    return permutations(range(1,rows+1))

def validate_perms(perms, deps):
    '''Returns the rows in perms which dont
    violate a dependency'''
    valid_perms = []
    for perm in perms:
        if check_perm(perm, deps) == True:
            valid_perms.append(perm)
    return valid_perms

def check_perm(perm, deps):
    '''Returns True or False according to whether
    perm violates a dependency'''
    prev_ind = [0]
    for index in perm:
        prev_ind.append(index)
        dep = deps[index-1, :]
        sub_true = all(elem in prev_ind for elem in dep)
        if sub_true is False:
            return False
        elif index == perm[-1]:
            return True

def sort_rates(times, probs):
    '''Returns an order list of the
    failure rates'''
    return (np.flip(np.argsort(
        1/times*np.log(1/(1-probs)))) + 1)
        
def get_expected_times(perms, df):
    '''Calculates the expected time according to the
    equation in appendix A'''
    sums = []
    for perm in perms:
        prob_col = df['probability'][np.array(perm)-1].values.astype(float)
        prob_comp = 1 - prob_col
        prob_mat = expand_to_lower_tri_mat(prob_comp)
        prod_arr = prob_prod_arr(prob_mat, prob_col)
        time_col = df['time'][np.array(perm) - 1].values.astype(float)
        time_mat = expand_to_lower_tri_mat(time_col)
        sum_arr = sum_times(time_mat)
        sums.append(expected_time_eq(prod_arr,
                                     sum_arr, time_col,
                                     prob_col, prob_comp))
    return sums


def expected_time_eq(prod_arr, sum_arr, time_col, prob_col,
                     prob_comp):
    prob_succ = np.prod(prob_comp)
    total_time = np.sum(time_col)
    failure_times_by_task = ((np.log((prob_comp)**(prob_comp)) 
                        + prob_col)
                        /np.log((1/prob_comp)**prob_col))
    for i,x in enumerate(failure_times_by_task):
        if prob_col[i]==0:
            failure_times_by_task[i] = 1/2
    exp_time_given_fail = np.sum(prod_arr*(sum_arr + time_col
                                           *failure_times_by_task))
    prob_fail = 1 - np.prod(prob_comp)
    return (prob_succ*total_time 
            + exp_time_given_fail*prob_fail)

def expand_to_lower_tri_mat(arr):
    '''Converts arr to a lower triangular matrix.
    e.g. [1,2,3] => [[1,0,0]
                     [1,2,0]
                     [1,2,3]]'''
    two_d_arr = np.expand_dims((arr), 0)
    square_mat = np.repeat(two_d_arr,
                             repeats=arr.shape[0],
                             axis=0)
    return np.tril(square_mat)
    
def prob_prod_arr(prob_mat, prob_col):
    '''Returns the product array that will satisfy the
    equation in the appendix'''
    np.fill_diagonal(prob_mat, 0)
    ones = np.triu(np.ones(np.shape(prob_mat)))
    np.fill_diagonal(ones, 0)
    return np.prod(ones + prob_mat + np.diag(prob_col), 1)

def sum_times(time_mat):
    '''Returns an array of sums from
    a lower triangular matrix'''
    np.fill_diagonal(time_mat, 0)
    return np.sum(time_mat, 1)

def adj_transposes(arr):
    """returns a matrix where each row
    is one of the n-1 adjacent transposes"""
    reps_mat = repeat_array_as_matrix(arr, len(arr)-1)
    for i in range(len(arr)-1):
        ith_row = reps_mat[i,:]
        ith_row[i], ith_row[i+1] = ith_row[i+1], ith_row[i]
    return reps_mat

def find_kids(row, arr, deps):
    '''Returns the inidicies of srtd_rates where
    the dependents of task in "row" are located'''
    child_rows = np.argwhere(deps==row)[:,0] + 1
    child_loc = np.where(np.in1d(arr, child_rows))[0]
    return child_loc

def repeat_array_as_matrix(arr, reps):
    '''e.g. array([1,2]) to array([[1,2],
                                    [1,2]])'''
    matrix_head = np.expand_dims(arr, axis=0)
    return np.repeat(matrix_head,
                      repeats=reps,
                        axis=0)

def find_parents(row, arr, deps):
    '''Returns the inidicies of arr where
    the tasks that task in row is dependent on
    are located'''
    dep = deps[row-1] #minus 1 cause index
    parent_rows = dep[np.nonzero(dep)] #rid zeros
    parent_loc = np.where(np.in1d(arr, parent_rows))[0]
    return parent_loc

def calc_disarray(arr, deps):
    '''returns a number according to how disarrayed
    arr is'''
    total_disarray = 0
    for task_loc, task in enumerate(arr):
        task_kids_loc = find_kids(task, arr, deps)
        task_pars_loc = find_parents(task, arr, deps)
        total_disarray += element_disarray(task_kids_loc,
                                           task_pars_loc,
                                           task_loc)
    return total_disarray

def find_least_disarrayed(mat, deps):
    '''Returns the rows of mat that have the
    lowest disarray score'''
    disarray_counts = np.zeros(mat.shape[0])
    for i, row in enumerate(mat):
        disarray_counts[i] = calc_disarray(row, deps)
    min_disarray = np.amin(disarray_counts)
    min_ind = np.where(disarray_counts==min_disarray)[0]
    return mat[min_ind,:]

def element_disarray(kid_loc, par_loc, loc):
    '''Returns a number according to how "out of place"
    an element is'''
    kids_to_left = np.extract(kid_loc<loc, kid_loc)
    pars_to_right = np.extract(par_loc>loc, par_loc)
    return (np.sum(loc - kids_to_left) + 
            np.sum(pars_to_right - loc))

def check_unique(mat, prev):
    '''Removes the rows in mat that
    are also in prev'''
    for i, row in enumerate(mat):
        if (prev==row).all(axis=1).any():
            mat[i,:] = np.zeros(
                mat.shape[1])
    mat = mat[~np.all(
        mat==0, axis=1)]
    return mat

def quick_best_order(arr, df, deps, count, prev):
    '''Returns an approximation for the optimal
    order.'''
    count += 1
    transposes = adj_transposes(arr)
    transposes = check_unique(transposes, prev)
    least_disarrayed = find_least_disarrayed(transposes, deps)
    expected_times = get_expected_times(least_disarrayed, df)
    best = least_disarrayed[np.argmin(expected_times)]
    prev = np.append(prev, np.expand_dims(best, 0), 0)
    if check_perm(best, deps)==True:
        return best
    elif count<50:
        return quick_best_order(best, df, deps, count, prev)
    else:
        return ("Error: Make sure dependencies don't produce"
                + " a contradiction.")

def present_list(arr, df):
    name_list = []
    index_to_name = dict(zip( df.index + 1, df['task']))
    for index in arr:
        name_list.append(index_to_name[index])
    return pd.DataFrame(name_list, columns=["Tasks"])

    
def plot_fig(sorted_sums):
    fig, axs = plt.subplots()
    axs.plot(np.arange(len(sorted_sums)), sorted_sums, color='black', ls='--')
    plt.xticks([])
    axs.set_title('Distribution of Expected Time')
    plt.show()

def main(df):
    df = switch_from_names(df)
    deps = df.filter(like='d').values
    times = df['time'].values.astype(float)
    probs = df['probability'].values.astype(float)
    srtd_rates = sort_rates(times, probs)
    prev = np.zeros([1,df.shape[0]])
    rows = df.shape[0]
    if rows < 10:
        perms = gen_perms(df)
        valid_perms = validate_perms(perms, deps)
        sums = get_expected_times(valid_perms, df)
        sorted_sums = np.sort(sums)
        best_order = valid_perms[np.argmin(sums)]
        plot_fig(sorted_sums)
        printmd("##### the optimal order is ")
        display(present_list(best_order, df))
        printmd("##### and the expected time is")
        printmd("##### " + str(sorted_sums[0]) + ".")

    else:
        best = quick_best_order(srtd_rates, df, deps, 0, prev)
        print('The optimal order is') 
        display(present_list(best, df))
        print("and the expected time is")
        display(get_expected_times(
                    np.expand_dims(best, 0), df)[0])

## Data Entry

In [6]:
myupload = ipywidgets.FileUpload(accept='.csv', multiple=False)
display(myupload)

FileUpload(value={}, accept='.csv', description='Upload')

In [10]:
sheet_butt = ipywidgets.Button(description='Build Sheet')
out = ipywidgets.Output()
sheet = build_sheet()

def sheet_butt_func(_):
    global sheet
    with out:
        try:
            clear_output()
            uploaded_filename = next(iter(myupload.value))
            content = myupload.value[uploaded_filename]['content']
            with open('task_sheet.csv', 'wb') as f: f.write(content)
            sheet = build_sheet()
            display(sheet)
        except:
            sheet = build_sheet()
            display(sheet)
sheet_butt.on_click(sheet_butt_func)
display(ipywidgets.VBox([sheet_butt,out]))

In [5]:
run = ipywidgets.Button(description='Run')
out = ipywidgets.Output()
def run_program(_):
    with out:
        df = df_from_sheet(sheet)
        df.to_csv('task_sheet.csv', index=False)
        local_file = FileLink('./task_sheet.csv', result_html_prefix="Click here to download the sheet: ")
        return main(df), display(local_file)
        
run.on_click(run_program)
buttons = ipywidgets.HBox([run])
ipywidgets.VBox([buttons,out])

## Appendix A

We want to calculate the expected time to termination for a given sequence of $n$ tasks. We either terminate by completing all of our tasks or failing on one of them. Let $T$ be the random variable denoting time until termination. $S$ be a random variable indicating whether we have succeeded or not.
From the law of total expectation, 

$$
\text{E}[T] = \text{E}[\text{E}[T \vert S]] = \text{P}(S = 1)\text{E}[T \vert S = 1] + \text{P}(S = 0)\text{E}[T \vert S = 0].
$$

To start we will focus our calculation on $\text{E}[T \vert S = 0]$. Let $L_i$ be the event that we fail on the $i^{\text{th}}$ task, for $i \in \{ 1, \dots, n\}$. Observe that the possible events given we fail are $L_1, \dots, L_n$. Thus, $L_1, \dots, L_n$ partitions $\{S = 0\}$. This means that we can apply the law of total expectation again to yield


$$
\text{E}[T \vert S = 0] = \text{P}(L_1)\text{E}[T \vert L_1] + \dots + \text{P}(L_n)\text{E}[T \vert L_n].
$$

$\text{P}(L_i)$ is determined by the probability of getting to task $i$ and the probability of failing at task $i$. It can be calculated with

$$
\text{P}(L_i) = p_i \prod_{k = 1}^{i - 1} (1 - p_{k}).
$$

Next, to calculate $\text{E}[T \vert L_i]$, let the starting time $t_{s}$ denote the time at which we start the $i^{\text{th}}$ task. Let $t_{i}$ denote the time the $i^{\text{th}}$ task takes us. Let $T_{i}$ be the random variable denoting the time spent on task $i$ given that we fail on task $i$. Suppose that the task fails at a rate $\lambda$ which is independent of how long we have been working on it. Then $f(t) = \lambda e^{-\lambda t}$ is the probability density function for failure time if the task is allowed to continue indefinately until we fail. We have assumed that the task will fail before $t_{i}$. It therefore follows that the probability density function for $T_{i}$ is $f(t \vert t< t_{i})$. This function has is given by

$$
f(t \vert t \leq t_{i})= \begin{cases}
\frac{1}{A}\lambda e^{-\lambda t}, \hspace{1cm} 0 \leq t \leq t_{i}, \\
0, \hspace{1.8cm} \text{otherwise}.
\end{cases}
$$

Where $A$ denotes the area under the curve of $f(t)$ from $0$ to $t_{i}$. To calculate $A$, we have

$$
A = \int_0^{t_{i}} \lambda e^{-\lambda t} dt = 1 - e^{-\lambda t_{i}}.
$$

To calculate $\text{E}[T]$ we now have

$$
\text{E}[T \vert L_i] = t_0 + \frac{1}{ 1 - e^{-\lambda t_{i}}} \int_0^{t_{i}} \lambda t e^{-\lambda t} dt,
$$

which after some simplification yeilds

$$
\text{E}[T \vert L_i] = t_0 + \frac{1 - t_{i} \lambda e^{-\lambda t_{i}} - e^{-\lambda t_{i}}}{\lambda(1 - e^{-\lambda t_{i}})}.
$$

$\lambda$ is currently an unknown value. The values we are given are $t_i$ and the probability of failure $p_i$. We need to write $\lambda$ in terms of these known quantities. This can be done with the following equation.

$$
1 - e^{- \lambda t_i} = p_{i},
$$

which yields

$$
\lambda = \frac{1}{t_i}\ln \Big{(}\frac{1}{1 - p_{i}}\Big{)}
$$

By substituting this value of $\lambda$ into the above equation and simplifying we get

$$
\text{E}[T \vert L_i] = t_0 + \frac{t_i\big{(}\ln\big{(}(1 - p_i)^{1 - p}\big{)} + p_i\big{)}}{\ln\Big{(}\big{(}\frac{1}{1-p_{i}}\big{)}^{p_{i}}\Big{)}}.
$$

Thus, $\text{P}(L_i)\text{E}[T \vert L_i]$ is given by

$$
\text{P}(L_i)\text{E}[T \vert L_i] = p_i \prod_{k = 1}^{i - 1} (1 - p_{k}) \Bigg{(}t_0 + \frac{t_i\big{(}\ln\big{(}(1 - p_i)^{1 - p}\big{)} + p_i\big{)}}{\ln\Big{(}\big{(}\frac{1}{1-p_{i}}\big{)}^{p_{i}}\Big{)}} \Bigg{)}.
$$

Note that for each $i \in \{1, \dots n\}$, $t_0$ is the time to complete the $1^{\text{st}}$ through $i - 1^{\text{th}}$ task. Then we can write $t_0$ as

$$
t_0 = \sum_{j = 1}^{i - 1} t_j.
$$

This implies that our final equation for $\text{P}(L_i)\text{E}[T \vert L_i]$ is given by

$$
\text{P}(L_i)\text{E}[T \vert L_i] = p_i \prod_{k = 1}^{i - 1} (1 - p_{k}) \Bigg{(}\sum_{j = 1}^{i - 1} t_j + \frac{t_i\big{(}\ln\big{(}(1 - p_i)^{1 - p}\big{)} + p_i\big{)}}{\ln\Big{(}\big{(}\frac{1}{1-p_{i}}\big{)}^{p_{i}}\Big{)}} \Bigg{)}.
$$

To find $\text{E}[T \vert S = 1]$, we must simply sum through all of the terms.

$$
\text{E}[T \vert S = 1] = \sum_{i = 1}^n \Bigg{(}p_i \prod_{k = 1}^{i - 1} (1 - p_{k}) \Bigg{(}\sum_{j = 1}^{i - 1} t_j + \frac{t_i\big{(}\ln\big{(}(1 - p_i)^{1 - p}\big{)} + p_i\big{)}}{\ln\Big{(}\big{(}\frac{1}{1-p_{i}}\big{)}^{p_{i}}\Big{)}} \Bigg{)} \Bigg{)}
$$

Next, by looking at the complement of $\text{P}(S = 0)$ we have

$$
\text{P}(S = 0) = 1 - \text{P}(S = 1) = 1 - \prod_{i = 1}^{n} (1 - p_i).
$$

$\text{E}[T \vert S = 1]$ is simply the expected time to success, which is simply the sum of the expected times to complete each task given by

$$
\text{E}[T \vert S = 1] = \sum_{i = 1}^n t_i
$$

Putting all of this together, we find

$$
\text{E}[T] = \Big{(}1 - \prod_{i = 1}^{n} (1 - p_i)\Big{)}\sum_{i = 1}^n t_i + \prod_{i = 1}^{n} (1 - p_i) \Bigg{(} \sum_{i = 1}^n \Bigg{(} p_i \prod_{k = 1}^{i - 1} (1 - p_{k}) \Bigg{(}\sum_{j = 1}^{i - 1} t_j + \frac{t_i\big{(}\ln\big{(}(1 - p_i)^{1 - p}\big{)} + p_i\big{)}}{\ln\Big{(}\big{(}\frac{1}{1-p_{i}}\big{)}^{p_{i}}\Big{)}} \Bigg{)} \Bigg{)} \Bigg{)}
$$